* 자연어 처리에서 필수적으로 사용되는 단어의 표현 방법인 원-핫 인코딩(One-hot encoding)과 워드 임베딩(Word Embedding)에 대해서 학습
### 1) NLP에서의 원-핫 인코딩(One-hot encoding)

In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4MB 55.0MB/s 
     |████████████████████████████████| 92kB 7.1MB/s 
     |████████████████████████████████| 460kB 32.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Okt
okt = Okt()
token = okt.morphs("나는 자연어 처리를 배운다")
print(token)

['나', '는', '자연어', '처리', '를', '배운다']


In [ ]:
word2index = {}
for voca in token:
  if voca not in word2index.keys():
    word2index[voca] = len(word2index)
print(word2index)

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


In [ ]:
def one_hot_encoding(word, word2index):
  onehot_vector = [0]*(len(word2index))
  index = word2index[word]
  onehot_vector[index] = 1
  return onehot_vector

In [ ]:
one_hot_encoding("자연어", word2index)

[0, 0, 1, 0, 0, 0]

### 2) 워드 임베딩
* 원핫 인코딩은 코퍼스의 단어가 100000개라면 벡터 차원도 100000이었어야함. + 고차원 벡터이기에 메모리 낭비 + 유사도 확인 불가
* 그러나 밀집 벡터 이용시 벡터 차원이 조밀해짐
* 워드 임베딩(단어를 밀집 벡터의 형태로 표현한 방법)
#####  Word2Vec : CBOW vs Skip-gram
* CBOW : 주변단어를 갖고 중간 단어를 예측 하는 방법 
* Skip-gram : 중간단어를 갖고  주변단어 예측를 하는 방법 

In [ ]:
import torch

In [ ]:
dog = torch.FloatTensor([1, 0, 0, 0, 0])
cat = torch.FloatTensor([0, 1, 0, 0, 0])
computer = torch.FloatTensor([0, 0, 1, 0, 0])
netbook = torch.FloatTensor([0, 0, 0, 1, 0])
book = torch.FloatTensor([0, 0, 0, 0, 1])

In [ ]:
print(torch.cosine_similarity(dog, cat, dim=0))
print(torch.cosine_similarity(cat, computer, dim=0))
print(torch.cosine_similarity(computer, netbook, dim=0))
print(torch.cosine_similarity(netbook, book, dim=0))

tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)


### 4) 영어/한국어 Word2Vec 훈련시키기

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f98de581a50>)

In [ ]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
# 저자의 경우 윈도우 바탕화면에서 작업하여서 'C:\Users\USER\Desktop\ted_en-20160408.xml'이 해당 파일의 경로.  
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.

In [ ]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [ ]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


In [ ]:
from gensim.models import Word2Vec, KeyedVectors
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)
'''
size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
window = 컨텍스트 윈도우 크기
min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
workers = 학습을 위한 프로세스 수
sg = 0은 CBOW, 1은 Skip-gram.
'''

'\nsize = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.\nwindow = 컨텍스트 윈도우 크기\nmin_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)\nworkers = 학습을 위한 프로세스 수\nsg = 0은 CBOW, 1은 Skip-gram.\n'

In [ ]:
model_result = model.wv.most_similar("boy")
print(model_result)

[('girl', 0.923625111579895), ('kid', 0.8343952894210815), ('lady', 0.8081405758857727), ('woman', 0.7867289185523987), ('man', 0.7692819833755493), ('mary', 0.759153425693512), ('sister', 0.7238417863845825), ('brother', 0.7158746719360352), ('baby', 0.7107657194137573), ('guy', 0.702812135219574)]


In [ ]:
model.wv.save_word2vec_format('./eng_w2v')
load_model = KeyedVectors.load_word2vec_format('eng_w2v')

In [ ]:
model_result = load_model.most_similar("boy")
print(model_result)

[('girl', 0.923625111579895), ('kid', 0.8343952894210815), ('lady', 0.8081405758857727), ('woman', 0.7867289185523987), ('man', 0.7692819833755493), ('mary', 0.759153425693512), ('sister', 0.7238417863845825), ('brother', 0.7158746719360352), ('baby', 0.7107657194137573), ('guy', 0.702812135219574)]


In [ ]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('파일 경로', binary=True)

In [ ]:
print(model.vectors.shape)

In [ ]:
print (model.similarity('this', 'is')) # 두 단어의 유사도 계산하기
print (model.similarity('post', 'book'))

### 6). 글로브(GloVe)
* 카운트 기반과 예측 기반을 모두 사용하는 방법론
* 예측기반의 Word2Vec의 단점을 보안한다는 목적을 가짐
* LSA(카운트/ 통계 기반) : 통계적 정보를 고려하지만 단어 의미와 유추작업에는 부족함
* Word2Vec(예측 기반) : 유추작업에는 뛰어나지만, 윈도우 크기 내에서만 주변 단어를 고려하기에 코퍼스 전체적인 통계 정보를 반영하지 못함
* => '임베딩 된 중심 단어와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 확률이 되도록 만드는 것'

### 7) 파이토치(PyTorch)의 nn.Embedding()

In [ ]:
import torch
import torch.nn as nn

In [ ]:
train_data = "you need to know how to code"

In [ ]:
word_set = set(train_data.split()) # set : 중복된 단어를 제거하여 집합 생성
word_set

{'code', 'how', 'know', 'need', 'to', 'you'}

In [ ]:
vocab = {word: i+2 for i, word in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1
print(vocab)

{'need': 2, 'know': 3, 'to': 4, 'code': 5, 'how': 6, 'you': 7, '<unk>': 0, '<pad>': 1}


In [ ]:
embedding_table = torch.FloatTensor([
                               [ 0.0,  0.0,  0.0],
                               [ 0.0,  0.0,  0.0],
                               [ 0.2,  0.9,  0.3],
                               [ 0.1,  0.5,  0.7],
                               [ 0.2,  0.1,  0.8],
                               [ 0.4,  0.1,  0.1],
                               [ 0.1,  0.8,  0.9],
                               [ 0.6,  0.1,  0.1]])

In [ ]:
sample = "you need to run".split()
index = []

for word in sample:
  try:
    index.append(vocab[word])
  except KeyError:
    index.append(vocab['<unk>'])

index = torch.LongTensor(index)

lookup_result = embedding_table[index, :]
print(index)
print(lookup_result)

tensor([7, 2, 4, 0])
tensor([[0.6000, 0.1000, 0.1000],
        [0.2000, 0.9000, 0.3000],
        [0.2000, 0.1000, 0.8000],
        [0.0000, 0.0000, 0.0000]])


In [ ]:
train_data = "you need to know how to code"
word_set = set(train_data.split())

vocab = {word: i+2 for i, word in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1

print(vocab)

{'need': 2, 'know': 3, 'to': 4, 'code': 5, 'how': 6, 'you': 7, '<unk>': 0, '<pad>': 1}


In [ ]:
embedding_layer = nn.Embedding(num_embeddings=len(vocab),
                               embedding_dim=3,
                               padding_idx=1)
'''
num_embeddings : 임베딩을 할 단어들의 개수. 다시 말해 단어 집합의 크기입니다.
embedding_dim : 임베딩 할 벡터의 차원입니다. 사용자가 정해주는 하이퍼파라미터입니다.
padding_idx : 선택적으로 사용하는 인자입니다. 패딩을 위한 토큰의 인덱스를 알려줍니다.
'''

'\nnum_embeddings : 임베딩을 할 단어들의 개수. 다시 말해 단어 집합의 크기입니다.\nembedding_dim : 임베딩 할 벡터의 차원입니다. 사용자가 정해주는 하이퍼파라미터입니다.\npadding_idx : 선택적으로 사용하는 인자입니다. 패딩을 위한 토큰의 인덱스를 알려줍니다.\n'

In [ ]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.4610, -0.3534, -0.3346],
        [ 0.0000,  0.0000,  0.0000],
        [-0.3596, -1.6494,  0.6563],
        [ 0.3017, -1.2605,  0.2688],
        [ 1.1220, -0.2450, -0.7597],
        [ 0.3690,  0.5782, -1.2220],
        [ 0.2117,  0.0300,  2.8962],
        [-0.4455,  1.2290, -0.3402]], requires_grad=True)


### 8) 사전 훈련된 워드 임베딩(Pretrained Word Embedding)

In [3]:
import torch
import torch.nn as nn

from torchtext.legacy import data, datasets

In [34]:
# 두개의 Field 객체 정의
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

In [ ]:
train_set, test_set = datasets.IMDB.splits(TEXT, LABEL)

In [28]:
print('훈련 데이터의 크기 : {}' .format(len(train_set)))

훈련 데이터의 크기 : 25000


In [9]:
print('테스트 데이터의 크기 : {}' .format(len(test_set)))

테스트 데이터의 크기 : 25000


In [10]:
print(vars(train_set[0]))

{'text': ['i', 'have', 'seen', 'the', 'trailer', 'for', 'this', 'movie', 'several', 'times', 'over,', 'and', 'i', 'have', 'to', 'say', 'that', 'ned', 'kelly', 'looks', 'like', 'it', 'is', 'going', 'to', 'be', 'a', 'wonderful', 'film.', 'when', 'i', 'saw', 'the', 'trailer', 'for', 'the', 'first', 'time,', 'i', 'could', 'not', 'take', 'my', 'eyes', 'away', 'from', 'it', '(it', 'got', 'my', 'attention', 'for', 'sure).', 'heath', 'ledger', 'sticks', 'to', 'what', 'he', 'knows', 'and', 'what', 'works', 'for', 'him,', 'period', 'pieces.', 'not', 'to', 'mention', 'orlando', 'bloom', ',who', 'is', 'seen', 'for', 'a', 'split', 'second', 'looks', 'fantastic.', 'i', 'think', 'that', 'this', 'movie', 'will', 'be', 'a', 'hit,', 'and', 'will', 'be', 'seen', 'over', 'and', 'over', 'again', 'my', 'many', 'people.'], 'label': 'pos'}


In [13]:
#Field 객체의 build_vocab을 통해 사전 훈련된 워드 임베딩을 사용할 수 있음

In [14]:
from gensim.models import KeyedVectors

In [ ]:
word2vec_model = KeyedVectors.load_word2vec_format('eng_w2v')

In [ ]:
print(word2vec_model['this']) 

In [16]:
from torchtext.vocab import GloVe

In [32]:
TEXT.build_vocab(train_set, vectors=GloVe(name='6B', dim=300), max_size=10000, min_freq=10)
LABEL.build_vocab(train_set)

In [23]:
print(TEXT.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f9a34848a10>>, {'<unk>': 0, '<pad>': 1})


In [33]:
print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT.vocab.vectors.shape))

임베딩 벡터의 개수와 차원 : torch.Size([2, 300]) 


In [25]:
print(TEXT.vocab.vectors[0])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [26]:
print(TEXT.vocab.vectors[1]) 

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [27]:
embedding_layer = nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze=False)